# Preparation

### What are the parameters for LTE throughput calculation? 

The parameters for LTE throughput calculation are the Signal to Noise Ratio (SNR) and the available bandwidth.

### Derive an equation to calculate throughput.

Shannon's equation describes the maximum rate at which information can be correctly received from a channel with a given bandwidth and SNR

In [70]:
const shannonsEquation = (snr: number, bandwidth: number) => 
    bandwidth * Math.log2(1 + snr)

However, this just is the bound and relies on a few assumptions, like infinite codewords, a random codebook and an AWGN channel. 

To calculate the possible throughput for LTE we have to multiply the available bandwidth with the efficiency of the chosen modulation and code rate. These values are chosen based on the CQI according to this table from the specification:

# ![A table](another-figure.png)

In [71]:
// Values from the table above
const efficiencyValues = [
  0, 0.1523, 0.2344, 0.377, 0.6016, 0.877, 1.1758, 1.4766, 1.9141, 2.4063,
  2.7305, 3.3223, 3.9023, 4.5234, 5.1152, 5.5547,
];

const getEfficiency = (cqi: number) => {
  const efficiency = efficiencyValues[cqi];
  if (efficiency === undefined) {
    throw new Error("CQI must be an integer between 0 and 15");
  }
  return efficiency;
};

NOTE: There are also other tables, but we choose this one.

To index the table we need the CQI value. The CQI is based on the SNR, we use the the function from the exercise sheet to convert them.

# ![A table](that-figure.png)

In [72]:
const snrToCqi = (snrDb: number) => {
    const cqi = snrDb*0.525+4.5;
    const clampedCqi = Math.max(0, Math.min(15, cqi));
    const clampedAndRoundedCqi = Math.round(clampedCqi)
    return clampedAndRoundedCqi
}

When we combine these two, we can calculate the throughput in based on the SNR and the bandwidth for LTE

In [73]:
const lteThroughput = (snrDb: number, bandwidth: number)  => 
    bandwidth * getEfficiency(snrToCqi(snrDb))

console.log(`The bandwidth for 20mHz and a SNR of 7dB is ${lteThroughput(7, 20e6)} bits per second`)

The bandwidth for 20mHz and a SNR of 7dB is 38282000 bits per second


If we compare that function with the channel capacity we can see that it is always slightly lower:

In [74]:
import vl from "npm:vega-lite-api"

type PlotConfig = {
    /** Plot a single function */
    fn?: (x: number) => number,
    /** Plot a multiple functions */
    fns?: Array<[string, (x: number) => number]>

    from?: number,
    to?: number,
    step?: number,
    xName?: string,
    yName?: string,
    /** Property name for the functions. */
    colorName?: string,
}
const generatePlotData = ({from = 0, to = 10, step = 0.1, xName = "x", yName = "y", fns = [], fn, colorName= "function"}: PlotConfig): Array<Record<string, number | string>> =>  {
  const length = Math.ceil((to - from) / step)
  const xPositions = Array.from({length}, (_, index) => from + index * step)

  const functions:  Array<[string, (x: number) => number]> =  fn ? [["default", fn]] : fns
  return xPositions.flatMap((x) => functions.map (([name,mapXToY]) =>
  ({
    [xName]: x,
    [yName]: mapXToY(x),
    ...(fn ? {} : {
      [colorName]: name
    })
  })))
}

const dbToFactor = (db: number) => 
    Math.pow(10, db/10)

const bandwidth = 20e6

const data = generatePlotData({
  from: -15,
  to: 25,
  step: 0.2,
  xName: "SNR (dB)",
  yName: "Bits per Second",
  colorName: "Type",
  fns: [
    ["Channel capacity", (snrDb: number) => shannonsEquation(dbToFactor(snrDb), bandwidth)],
    ["LTE throughput", (snrDb: number) => lteThroughput(snrDb, bandwidth)]
  ]
})

await vl.markLine()
.data(data)
.encode(
  vl.x().fieldQ("SNR (dB)"), 
  vl.y().fieldQ("Bits per Second"),
  vl.color().field("Type"),
)
.width(700)
.height(400)

# LTE throughput at the link layer

In [75]:
const measurementsExerciseTwo = [
    {bandwidth: 20, snr: 8.4},
    {bandwidth: 10, snr: 11.8},
    {bandwidth: 20, snr: 8.6},
    {bandwidth: 10, snr: 12.4},
    {bandwidth: 20, snr: 7.4},
]

# Interference in LTE

In [ ]:
const measurementsExerciseThree = [
    {rsrp: -103, snr: 7.6, bandwidth: 20, neighbours: [
        {id: 51, rsrp: -112},
        {id: 53, rsrp: -108},
        {id: 485, rsrp: -111},
    ]},
    {id: 262, rsrp: -100, snr: 8.2, bandwidth: 20, neighbours: [
        {id: 51, rsrp: -113},
        {id: 53, rsrp: -115},
        {id: 485, rsrp: -113},
        {id: 331, rsrp: -114},
    ]},
    {id: 262, rsrp: -96, snr: 9.8, bandwidth: 20, neighbours: [
        {id: 326, rsrp: -108},
        {id: 155, rsrp: -109},
        {id: 429, rsrp: -103},
    ]},
    {id: 262, rsrp: -99, snr: 8.4, bandwidth: 20, neighbours: [
        {id: 51, rsrp: -109},
        {id: 485, rsrp: -113},
    ]},
    {id: 262, rsrp: -100, snr: 7.2, bandwidth: 20, neighbours: [
        {id: 51, rsrp: -113},
        {id: 53, rsrp: -112},
        {id: 253, rsrp: -113},
    ]},
]